아래와 같은 CLI를 통해서 서비스를 생성할 수 있습니다. 
이번 실습은 [다음 문서](http://azure.studydev.com/openai/aoai_2024_pe_02.pdf)의 p.47~p.55 제공하는 스크린샷 이미지를 바탕으로 Azure Portal에서의 리소스 접근 및 GPT 모델을 생성하여 진행합니다. 

- 생성에 사용한 Azure OpenAI Endpoint URL, Azure OpenAI API Key, GPT 모델 배포 이름(deployment_id)을 .env 파일에 저장합니다.
- .env 파일이 존재하지 않을 경우, .env.sample 파일을 복사하여 이름을 바꾸어 생성합니다.
- 아래 명령은 CLI를 이용하여 진행하는 예시이며, 개발자 분들은 여기서 [다음으로 넘어 갑시다.](./02_OpenAI_getting_started.ipynb)
- 2024-12-10 업데이트 하였습니다. (주요 모델 최신 버전으로 변경, 코드 수정)

# Prompt Engineering Workshop 진행을 위한 리소스 생성 가이드 - CLI 버전
아래 가이드 문서는 회사 규모에서 대규모의 Prompt Engineering Workshop 교육을 할 때 미리 인프라 준비를 위한 과정입니다.  
2일 이상 진행할 때, 10여명 이상의 개발자들이 해커톤을 진행할 때 사전에 리소스를 생성하는 방법과 권한 관리를 하는 방법에 대한 가이드입니다.  

***아래 CLI 코드는 워크샵을 준비해 주시는 Infra 담당자 분이 실행하는 코드입니다.***  
일반 개인의 경우, 아래와 같이 리소스를 대규모로 생성할 필요는 없습니다.  

## 1. 구독 생성 및 AOAI 승인
- Azure 계정을 생성합니다.
- Subscription(구독) 생성합니다. (팀 수 = 구독 * 리전 개수)
  - 준비할 구독 수는 다음과 같이 산정할 수 있습니다. 구독을 미리 여러 개 만드는 이유는 Quotas(API 할당량)에 대한 제약으로 자유로워지기 위함 입니다.
  - 예를 들어, 참석 인원이 75명이고 15개팀으로 나뉠 경우, 1개의 구독과 아래와 같이 10개 리전(eastus, eastus2, westus3, canadaeast, uksouth, swedencentral, francecentral, norwayeast, japaneast, southindia) 10개 + 5개 리소스를 생성합니다.
  - 예시: 10(팀) = 1(구독) * 10(리전), 20(팀) = 1(구독) * 10(리전) * 2, 30(팀) = 1(구독) * 10(리전) * 3
  - 30팀이 넘어갈 경우, 구독 개수를 추가하는 형태로 진행할 수 있습니다.

## 2. 구독별 리소스 그룹과 리소스를 사전 생성
- 구독별 OpenAI 사용 승인이 나면, 워크샵과 프롬프톤 진행 시간을 절약하기 위해 리소스 그룹(Resource Group)과 리소스(azure OpenAI Resource)를 사전에 생성합니다. 
  - 생성된 리소스에 행사 참여하는 사용자가 정해진 리소스에 접근할 수 있도록 미리 RBAC 매핑이 필요합니다.
  - 워크샵 기간 동안에는 참석하는 사용자분들의 서비스 접근 권한 제어는 [Contributer(기여자)](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles#contributor) 기반으로 해당 Subscription ID 별로 권한 부여하는 것을 추천 드립니다. (워크샵 종료후 모두 삭제, 권한 부여는 최소 권한만 부여하는 것이 바람직합니다.)
  - 다음의 리전을 활용합니다. `text-embedding-3-large`와 `GPT-4o`, `GPT-4o mini` 를 동시 지원하는 리전인 아래 10개를 추천합니다. (기준일 2024-12-10)
    - eastus, eastus2, westus3, canadaeast, uksouth, swedencentral, francecentral, norwayeast, japaneast, southindia

- **CLI를 이용한 리소스 생성 예시**
네이밍 규칙은 회사 규정에 따라서 정하시면 됩니다. 위에서 팀별로 기 생성해둔 각각의 구독에 대해서 실행하는 예시입니다.
해당 명령은 Azure Portal 상단 검색창 우측에 있는 `Cloud Shell (PowerShell)`을 실행하여 시작할 수 있습니다.

이후 CLI 명령은 Jupyter Notebook에서 커스터마이징 하여 실행하는 것을 추천 드립니다.

| 순서 | 작업명 | CLI 명령어 |
|---|---|---|
|01|활성 테넌트를 확인|```az account show```|
|02|작업 구독 설정|```az account set --subscription "xxxxxxxxx-xxxxx-xxxx-xxxx-xxxxxxxxxxx"```|
|03|그룹 리소스 생성|```az group create --name companython-2024-rg --location eastus```|
|04|OpenAI 리전 리소스 생성 01|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-01 --custom-domain companython-2024-pt-01 --kind OpenAI --sku s0 -l eastus```|
|05|OpenAI 리전 리소스 생성 02|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-02 --custom-domain companython-2024-pt-02 --kind OpenAI --sku s0 -l eastus2```|
|06|OpenAI 리전 리소스 생성 03|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-03 --custom-domain companython-2024-pt-03 --kind OpenAI --sku s0 -l westus3```|
|07|OpenAI 리전 리소스 생성 04|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-04 --custom-domain companython-2024-pt-04 --kind OpenAI --sku s0 -l canadaeast```|
|08|OpenAI 리전 리소스 생성 05|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-05 --custom-domain companython-2024-pt-05 --kind OpenAI --sku s0 -l uksouth```|
|09|OpenAI 리전 리소스 생성 06|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-06 --custom-domain companython-2024-pt-06 --kind OpenAI --sku s0 -l swedencentral```|
|10|OpenAI 리전 리소스 생성 07|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-07 --custom-domain companython-2024-pt-07 --kind OpenAI --sku s0 -l francecentral```|
|11|OpenAI 리전 리소스 생성 08|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-08 --custom-domain companython-2024-pt-08 --kind OpenAI --sku s0 -l norwayeast```|
|12|OpenAI 리전 리소스 생성 09|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-09 --custom-domain companython-2024-pt-09 --kind OpenAI --sku s0 -l japaneast```|
|13|OpenAI 리전 리소스 생성 10|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-10 --custom-domain companython-2024-pt-10 --kind OpenAI --sku s0 -l southindia```|


***Custom 필요!*** - 구독별로 아래 `prompthon_name`이나 `prompthon_team_name`의 prefix 이름은 변경합니다. 이때 두 개의 값은 Global로 중복이 일어나지 않는 값이 되도록 설정합니다.  
구독이 다수일 경우에는 3번이 종료되면 다시 새로운 구독으로 2번부터 반복 진행합니다.

In [6]:
company_event_name  ="companython-2024"

아래는 현재 디바이스에서 Azure CLI를 활용하기 위해 로그인 하는 방법에 대한 설명입니다. 

In [ ]:
!az config set core.login_experience_v2=off
!az login --use-device-code

In [ ]:
!az account show

***Custom 필요!*** - Prompthon에 사용할 구독(Subscription)의 `id` 값을 복사하여 변경한 후, 아래 명령을 실행합니다. (만약 위에서 이미 선택된 상태라면 수행할 필요가 없습니다.)

In [ ]:
!az account set --subscription "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

이번 워크샵 또는 해커톤에서 리소스를 통합 관리하기 위한 `Resource Group` 을 생성합니다. 이후 생성하는 모든 리소스는 이 `Resource Group` 기반으로 일괄 관리 할 수 있습니다.

In [ ]:
!az group create --name $company_event_name"-rg" --location eastus

구독당 리전별로 리소스를 생성합니다. (각 리전당 리소스는 생성을 위해 약 1분씩 시간이 걸립니다.)  
아래는 전체 25개 팀을 만들때 쓸 수 있는 예시이며, 구독당 30개까지만 리소스를 생성할 수 있습니다.

In [ ]:
total_resources = 25
region_list = ['eastus', 'eastus2', 'westus3', 'canadaeast', 'uksouth', 'swedencentral', 'francecentral', 'norwayeast', 'japaneast', 'southindia']
region_list = region_list * (total_resources // len(region_list)) + region_list[:total_resources % len(region_list)]

for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    create_openai_resource = (f'az cognitiveservices account create -g {company_event_name}-rg -n {company_event_name}-pt-{number} --custom-domain {company_event_name}-pt-{number} --kind OpenAI --sku s0 -l {region_list[i]}')
    !{create_openai_resource}

추가 구독(Subscription)이 더 있을 경우, 위 과정을 통해 다른 구독 계정 ID 기반으로 만들어 나가면 됩니다.

## 3. Azure OpenAI 모델 배포
위에서 생성된 각 Azure OpenAI 리소스에 각 모델을 배포합니다.  
3개 모델(text-embedding-3-large, gpt-4o, gpt-4o-mini)을 배포합니다.(2.4번부터 생성된 각 리소스별로 아래 3개 모델을 반복 배포합니다.)  
각 리전별 3개 모델 배포에 걸리는 시간은 약 10초입니다.

|순서|작업|CLI 명령|
|---|---|---|
|1|임베딩 API 배포|```az cognitiveservices account deployment create -name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name text-embedding-3-large --model-name text-embedding-3-large --model-version "1" --model-format OpenAI --sku-capacity "100" --sku-name "Standard"```|
|2|gpt-4o Global Standard 형태로 API 배포|```az cognitiveservices account deployment create --name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name gpt-4og --model-name gpt-4o --model-version "2024-08-06" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"```|
|3|gpt-4o mini Global Standard 형태로 API 배포|```az cognitiveservices account deployment create --name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name gpt-4o --model-name gpt-4o --model-version "2024-07-18" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"```|

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    text_embedding_model = (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name text-embedding-3-large --model-name text-embedding-3-large --model-version "1" --model-format OpenAI --sku-capacity "100" --sku-name "Standard"')
    gpt_4o_model =         (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o --model-name gpt-4o --model-version "2024-08-06" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"')
    gpt_4o_mini_model =    (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o-mini --model-name gpt-4o-mini --model-version "2024-07-18" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"')
    !{text_embedding_model}
    !{gpt_4o_model}
    !{gpt_4o_mini_model}

## 4. 사용자 등록  
Microsoft Entra ID에서 User를 등록합니다. 관리의 편의를 위해서 팀별 또는 해커톤 그룹을 만들어 관리할 수 있습니다.  
팀별로 Group을 만들고 위에서 만든 User를 Group에 넣습니다.  
- 다음 CLI를 통하여 사용자를 등록할 수 있습니다. [참고 문서 링크](https://learn.microsoft.com/en-us/cli/azure/ad/user?view=azure-cli-latest#az-ad-user-create)  
`az ad user create --display-name myuser --password password --user-principal-name myuser@contoso.com`

## 5. 리소스 권한 부여
사용자 등록과 더불어 참석하는 사용자를 Microsoft Entra ID에 등록하고, Azure OpenAI 및 AI Search, Blob Storage, AI Foundry에 접근할 수 있는 권한을 부여합니다.  
해커톤에서 빠르게 실험하고 테스트하기 위한여 사용자별로(그룹 기반 관리도 가능) ["Contributor"](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles#contributor) 권한을 구독에 부여하는 것을 권해 드립니다.

최소 권한 부여 원칙에 의거하여 운영하는 것이 모범 사례이나, 해커톤 끝남과 동시에 회수하는 것을 권해 드리고 있습니다.  
만약, **보안 모범 사례 기반으로 최소 권한 기반**으로 접근 가능하도록 구성한다면 다음과 같습니다.  
- 2~3번 과정에서 만든 Resource Groups의 IAM에 들어가서 다음의 Role을 각각 해당 Group에 부여합니다. (OpenAI, AI Search, Storage Account 권한)  
- 만약, Azure OpenAI Stduio에서 `Add your data` 기능을 활용하여 RAG 서비스 연결(AI Search와 Blob Storage 연결)을 쉽게 만들고자 할 경우에는 `구독 ID`에 Role을 Contributor로 등록하는 것을 추천 합니다. 이유는 AI Search 서비스를 만들고 나서 Blob storage로 접근할 수 있는 권한을 매번 설정하는 작업이 매우 번거로울 수 있습니다. [구조에 대한 이해](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/use-your-data-securely)

위와 같은 방법으로 구성할 경우, Subscription에 User 또는 Group 별로 Owner 권한을 주지 않더라도 Azure 서비스로 실습을 진행할 수 있습니다.  
이외의 서비스들을 활용하고자 할 경우에, OCR, Text-to-speech, Speech-to-Text, App Services(즉시 챗봇 서비스 배포), Promptflow를 활용할 수 있는 Azure AI Foundry 등 역시 각각의 Role을 찾아서 부여할 수 있지만, 해커톤 당일 수십명의 사용자가 각각 권한을 요청해 올 경우, 진행이 번거로울 수 있는 점을 전달 드립니다.  
  
여기까지 진행하면, 생성한 사용자 기반으로 각 리소스에 접근 및 서비스 활성화가 가능한지 사전에 로그인 해서 Azure OpenAI Playground에서 활용 가능한지 확인하면 준비가 완료됩니다.  
준비과정은 구독 생성으로부터 각 사용자별 접근 테스트까지 약 1~2주의 기간이 있다면 안정적입니다. 

## 6. 리소스 삭제 방법
리소스를 삭제는 생성의 역순으로 진행합니다.  
리소스를 삭제 하기 전에 Azure OpenAI 리소스에 배포한 모델들을 삭제하고 리소스 그룹을 하나씩 삭제하는 것을 추천 드립니다.  
모델 삭제 전, 리소스를 삭제할 경우, 해당 리소스에 배포되어 있는 모델의 Quotas가 잡혀 있는 것처럼 남는 문제가 있을 수 있습니다.

### 6.1 배포된 Azure OpenAI 모델 삭제

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    text_embedding_model = (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name text-embedding-3-large')
    gpt_4o_model =         (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o')
    gpt_4o_mini_model =    (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o-mini')
    !{text_embedding_model}
    !{gpt_4o_model}
    !{gpt_4o_mini_model}

### 6.2 배포된 Azure OpenAI 리소스 삭제  
각 리전별 리소스 삭제에는 약 10초 정도의 시간이 필요합니다. (반드시 위 배포되어 있는 모델을 삭제한 후 리소스를 삭제해 주세요.)

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    delete_openai_resource = (f'az cognitiveservices account delete -g {company_event_name}-rg -n {company_event_name}-pt-{number}')
    !{delete_openai_resource}